In [1]:
# importing libraries
import pandas as pd # data science essentials
import pulp 
from itertools import combinations

# loading data
teambuilding = pd.read_excel('./datasets/MM_EmployeeData.xlsx',sheet_name= "Data")

# global variables
max_teams = 5
max_team_size = 4
total_population_size = 20

# displaying the head of the dataset
teambuilding.head(n = 5)

,ID,Gender,Department,Nationality,Experience,Title
0,1023,Female,Employee Relations,France,33,Manager
1,1032,Female,Payroll,India,31,Manager
2,1072,Male,Collections,India,8,Junior
3,1188,Female,Recruitment,Pakistan,18,Senior
4,1347,Female,Advertising,Switzerland,30,Manager


In [2]:
#Setting the dataframe to the total population size so we can use it both for testing and the actual simulation (ie: 20 vs 100)
teambuilding = teambuilding.head(n=total_population_size)
teambuilding

,ID,Gender,Department,Nationality,Experience,Title
0,1023,Female,Employee Relations,France,33,Manager
1,1032,Female,Payroll,India,31,Manager
2,1072,Male,Collections,India,8,Junior
3,1188,Female,Recruitment,Pakistan,18,Senior
4,1347,Female,Advertising,Switzerland,30,Manager
5,1405,Male,Training,Panama,1,Junior
6,1407,Male,Benefits,Singapore,20,Senior
7,1424,Female,Security,Singapore,27,Senior
8,1487,Female,Presales,Iraq,10,Junior
9,1571,Female,Infrastructure,France,6,Junior


In [4]:
#create list of all possible teams 
possible_team = [
    #creating a list of every possible combinations, then specifying the max combination size to be 4
    tuple(c) for c in pulp.allcombinations(teambuilding['ID'], max_team_size)
    # we want each teams to be exactly made of 4 members, without the if statement, 
    # teams of 1, 2, 3 and 4 teams will be created
    if len(c) == max_team_size 
]

#get length of possible team
len(possible_team)

4845

In [ ]:
#check what's inside of possible team (list of all the possible team combinations)
possible_team

In [ ]:
#Definition of Penalty Function
def satisfaction(team_tuple):
    #gives the whole dataframe of this team (give it the ID and it looks up the rest of the columns)
    teamdf = teambuilding[(teambuilding['ID'].isin(team_tuple))] 
    penalty = 0  
    
    """
    constraint 1
    """
    #ratio for team
    teamdf_male_ratio = teamdf[teamdf['Gender'] == 'Male']['Gender'].count() / max_team_size
    #ratio for population
    population_male_ratio = teambuilding[teambuilding['Gender'] == 'Male']['Gender'].count() / total_population_size 
    
    
    # constraint1 is the absolute value(male ratio of team - male ratio of total population)
    constraint1 = abs(teamdf_male_ratio - population_male_ratio)
      
    """
    constraint 2 
    """
    #maximum team size - number of unique departments within a team

    constraint2 = max_team_size - teamdf['Department'].unique().size
    
    """
    constraint 3
    """
    #absolutevalue((average team experience)-(average population experience))
    constraint3 = abs(teamdf['Experience'].mean() - teambuilding['Experience'].mean())
   
    """
    constraint 4  
    """ 
    #maximum team size - number of unique Nationalities within a team
    constraint4 = max_team_size - teamdf['Nationality'].unique().size

    """
    constraint 5
    """
    #Counting the number of juniors or managers in a team
    junior_count = teamdf[teamdf['Title'] == 'Junior']['Title'].count()
    manager_count = teamdf[teamdf['Title'] == 'Manager']['Title'].count()
    
    #applying penalty if conditions are not met
    if junior_count == 0:
        penalty +=  1
    if manager_count == 0:
        penalty +=  1
        
    constraint5 = penalty
    
    #return normalized constraints to make each constraint' maximum value = 1
    #max value of each constraint has been calculated separately
    #constraint 3 has been normalized using values of 100 people and not the small samples that were used for testing
    return constraint1 + constraint2/3 + constraint3/22.88 + constraint4/3 + constraint5/2

In [ ]:
#manually test if the penalty function works by calling the penalty function on a possible team
satisfaction([1023,1032, 1072, 1188])

In [ ]:
#the simulation runs 3.9 million iterations and picks the best teams, to make sure this
# code works, we can test it on 20 choose 4 (4845 iterations) and outputs the best 5 teams, you would have to change 
# the total population size in the penalty function, global variable, max teams = 8 

#create a binary variable to state that a team setting is used
x = pulp.LpVariable.dicts('team', possible_team, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)


teambuilding_model = pulp.LpProblem("Team_Building_Model", pulp.LpMinimize)

teambuilding_model += pulp.lpSum([satisfaction(team_tuple) * x[team_tuple] for team_tuple in possible_team])

#specify the maximum number of teams
teambuilding_model += pulp.lpSum([x[team_tuple] for team_tuple in possible_team]) <= max_teams, \
                            "Maximum_number_of_teams"


#An employee must be assigned to one and only one team
for employee in teambuilding['ID'].tolist():
    teambuilding_model += pulp.lpSum([x[team_tuple] for team_tuple in possible_team
                                if employee in team_tuple]) == 1, "Must_commit_%s"%employee

teambuilding_model.solve()

print(f' The following is the best possible team Assignement  out of {len(possible_team)} possible team combinations.')
#print('The following team set assignement contains 25 unique teams with 100 unique employees')
df = pd.DataFrame({
    'team_combination': [pair[0] for pair in x.items() if pair[1].value() == 1.0],
    'penalty_score': [satisfaction(pair[0]) for pair in x.items() if pair[1].value() == 1.0]
})

df.sort_values(by = ['penalty_score'], inplace=True, ascending=True)
df